In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


In [4]:
from langchain_core.prompts import PromptTemplate

# 1. 템플릿 정의 ({country}가 변수)
template = "{country}의 수도는 어디인가요?"

# 2. 객체 생성
prompt_template = PromptTemplate.from_template(template)

# 3. 프롬프트 생성 예시
print(prompt_template.format(country="대한민국"))

대한민국의 수도는 어디인가요?


In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 프롬프트 생성
prompt = PromptTemplate.from_template("{country}의 수도는 어디인가요?")

# 2. 모델 생성
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

# 3. 출력 파서
output_parser = StrOutputParser()

# 4. Chain 연결 (LCEL)
chain = prompt | model | output_parser

# 실행
print(chain.invoke({"country": "대한민국"}))

c:\Users\jeongseunghyun\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-NQ-W-e1F-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


대한민국의 수도는 서울입니다.


In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4.1-nano",  # 모델명
)

answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!")

for token in answer:
    print(token.content, end="", flush=True)

물론입니다! 대한민국의 아름다운 관광지 10곳과 그 주소를 소개해 드리겠습니다.

1. 경복궁 (Gyeongbokgung Palace)  
서울특별시 종로구 사직로 161

2. 제주도 한라산 국립공원 (Hallasan National Park, Jeju Island)  
제주특별자치도 제주시 특별자치도 제주시 조천읍 한라산로 2070

3. 부산 해운대 해수욕장 (Haeundae Beach, Busan)  
부산광역시 해운대구 해운대해변로 264

4. 강원도 설악산 국립공원 (Seoraksan National Park)  
강원도 속초시 설악산로 833

5. 경상남도 통영 한산도 (Tongyeong Hansando)  
경상남도 통영시 한산면 죽림리 한산도

6. 전라남도 여수 오동도 (Oedo Island, Yeosu)  
전라남도 여수시 오동도길 61

7. 충청남도 공주 무령왕릉 (Royal Tomb of King Muryeong)  
충청남도 공주시 송산리 1-1

8. 전라북도 전주 한옥마을 (Jeonju Hanok Village)  
전라북도 전주시 완산구 기린대로 99

9. 경상북도 경주 불국사 (Bulguksa Temple, Gyeongju)  
경상북도 경주시 불국로 385

10. 강원도 강릉 경포대 (Gyeongpodae Pavilion, Gangneung)  
강원도 강릉시 경포로 365

이 외에도 대한민국에는 아름다운 곳이 많지만, 위의 장소들은 대표적이고 인기 있는 관광지입니다. 방문 계획에 참고하시기 바랍니다!

In [11]:
import asyncio

async def async_run():
    response = await chain.abatch([ 
        {"country": "대한민국"}, 
        {"country": "미국"}, 
        {"country": "프랑스"} 
    ])
    print(f"\n[LLM 답변 도착]: {response}")

async def main():
    task = asyncio.create_task(async_run())
    
    print("--- 숫자 세기 시작 ---")
    for i in range(10):
        print(i, end=" ")
        await asyncio.sleep(0.5) # 텀주기
    print("\n--- 숫자 세기 끝 ---")

    await task 

# 실행
await main()

--- 숫자 세기 시작 ---
0 1 2 
[LLM 답변 도착]: ['대한민국의 수도는 서울입니다.', '미국의 수도는 워싱턴 D.C.입니다.', '프랑스의 수도는 파리입니다.']
3 4 5 6 7 8 9 
--- 숫자 세기 끝 ---


In [14]:
from langchain_core.runnables import RunnableParallel

# Chain 1: 수도 질문
chain1 = (
    PromptTemplate.from_template("{country}의 수도는?")
    | model
    | StrOutputParser()
)

# Chain 2: 면적 질문
chain2 = (
    PromptTemplate.from_template("{country}의 면적은?")
    | model
    | StrOutputParser()
)

# 병렬 체인 결합
combined = RunnableParallel(capital=chain1, area=chain2)

# 실행
result = combined.invoke({"country": "대한민국"})
print(result)

{'capital': '대한민국의 수도는 서울입니다.', 'area': '대한민국의 면적은 약 100,210 평방킬로미터입니다. 이는 한반도의 남쪽 부분에 해당하며, 북한과 함께 한반도를 구성하고 있습니다.'}


In [17]:
from langchain_core.runnables import RunnablePassthrough

# {"num": 5} 대신 5만 넣어도 {"num": 5}로 변환되어 프롬프트로 전달됨
chain = {"num": RunnablePassthrough()} | PromptTemplate.from_template("{num}의 10배는?") | model

print(chain.invoke(5).content)

5의 10배는 50입니다.


In [18]:
# 입력값 x에 대해 x["num"] * 3 한 값을 'mult'라는 키로 추가
(RunnablePassthrough.assign(mult=lambda x: x["num"] * 3)).invoke({"num": 1})

# 결과: {'num': 1, 'mult': 3}

{'num': 1, 'mult': 3}

In [22]:
from langchain_core.runnables import RunnableLambda
from datetime import datetime

# 사용자 정의 함수를 RunnableLambda 에서 사용할 때 매개변수가 없으면 오류 반환
def get_today(x):
    # 매개변수와 상관없이 오늘 날짜 return
    return datetime.today().strftime("%b-%d")

# 체인 구성: 오늘 날짜를 가져와서 프롬프트에 주입
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    # today에 오늘 날짜, n은 invoke 시 전달받은 값
    | PromptTemplate.from_template("{today}가 생일인 대한민국의 유명인 {n}명을 알려줘")
    | model
    | StrOutputParser()
)

print(chain.invoke(2))
# prompt: 오늘 날짜(Dec-23) 가 생일인 대한민국의 유명인 2명을 알려줘

12월 23일에 생일인 대한민국의 유명인으로는 다음 두 명이 있습니다:

1. **이상민** - 대한민국의 가수이자 방송인으로, 그룹 '젝스키스'의 멤버로 잘 알려져 있습니다.
2. **김범수** - 대한민국의 가수로, 뛰어난 가창력으로 많은 사랑을 받고 있는 아티스트입니다.

이 외에도 12월 23일에 태어난 유명인들이 있을 수 있습니다.
